In [24]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
from sklearn.metrics import r2_score
from gpytorch.mlls import ExactMarginalLogLikelihood
import gpytorch
import altair as alt
import matplotlib.pyplot as plt

In [25]:
dft_data = pd.read_csv("/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/DFT_Data_clean_06_10.csv")
expV_data = pd.read_csv("/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP/Vext_allTEMP_expV_logBin_cut5_-15_100Bins.csv")
data = pd.merge(dft_data, expV_data, 'inner', on=["structure_name", "temperature_kelvin"])
feature_columns = [f"bin_{i}" for i in range(100)]
data = data[data.beladung_mol_per_kg > 0]
data = data[(data.temperature_kelvin == 400) & (data.pressure_bar == 0.1)]

In [26]:
data.head()

,structure_name,pressure_bar,temperature_kelvin,volume_kubAng,grid.dv,density_Atmos_per_kubAng,density_bulk,fraction_of_used_points,beladung_mol_per_kg,beladung_atoms,...,bin_92,bin_93,bin_94,bin_95,bin_96,bin_97,bin_98,bin_99,x_min,x_max
8,EON,0.1,400.0,3563.018109,0.013592,9.001273,0.000002,0.100189,0.033937,0.122344,...,0,0,0,0,0,0,0,0,0.006738,3.269017e+06
46,OKO,0.1,400.0,3884.589614,0.014819,6.396076,0.000002,0.120880,0.023198,0.094780,...,0,0,0,0,0,0,0,0,0.006738,3.269017e+06
84,MWW,0.1,400.0,4518.749756,0.017238,7.448088,0.000002,0.146805,0.029678,0.128388,...,0,0,0,0,0,0,0,0,0.006738,3.269017e+06
122,SFS,0.1,400.0,3367.496936,0.012846,5.940400,0.000002,0.134964,0.022680,0.076310,...,0,0,0,0,0,0,0,0,0.006738,3.269017e+06
160,DDR,0.1,400.0,6715.860313,0.025619,28.715224,0.000002,0.082912,0.098741,0.735655,...,0,0,0,0,0,0,0,0,0.006738,3.269017e+06


In [27]:
new_data = data.copy()
new_data["beladung_pro_volumen"] = new_data["beladung_atoms"] / new_data["volume_kubAng"]
new_data.head()

new_data[feature_columns] = (
    new_data[feature_columns]
    .multiply(new_data["grid.dv"], axis=0)
    #.div(new_data["volume_kubAng"], axis=0)
)

new_data[feature_columns].head()


,bin_0,bin_1,bin_2,bin_3,bin_4,bin_5,bin_6,bin_7,bin_8,bin_9,...,bin_90,bin_91,bin_92,bin_93,bin_94,bin_95,bin_96,bin_97,bin_98,bin_99
8,1.087347,1.304816,1.522286,1.413551,2.065959,2.174694,2.174694,1.957224,2.065959,1.739755,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,2.963707,1.541127,2.252417,1.185483,2.252417,2.133869,3.200803,2.608062,1.778224,2.252417,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84,2.964878,2.758026,3.516483,3.102779,5.516052,3.930187,3.792286,3.861236,2.482223,2.895927,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,2.312277,2.260893,2.415044,2.106741,2.517812,2.620580,2.466428,2.415044,2.106741,2.620580,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,2.971801,2.408184,2.536278,2.382565,3.304848,4.150274,2.869325,3.509799,2.971801,3.714751,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
